#Experimentos de dropout
Com **512, 1024** [N]euronios. **1, 2, 3** [L]ayers. **Sem, All, 1 Last Dense 1 e Last CNN, Só All Dense, Só Last Dense, Só All CNN, Só Last CNN** [D]ropout.

Parametros:
* Batch	64
* epocas	100
* LR	1,00E-05
* Optimizer	Adam

Parametros Early Stopping:
* Monitor	val_auc
* Min_delta	0
* Patience	5
* verbose	1
* mode	auto
* baseline	none
* restore_best_weights	True

# Imports e Installs

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Rescaling
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import TensorBoard

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 30.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Ajustando o DS

## Baixando o DS

In [ ]:
!kaggle datasets download -d pedroamaro/smaller-rsna-ds-train-test

100% 7.25G/7.25G [03:15<00:00, 59.5MB/s]
100% 7.25G/7.25G [03:15<00:00, 39.8MB/s]


In [ ]:
!unzip /content/smaller-rsna-ds-train-test.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/smaller-rsna-ds-train-test.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_train/no | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_train/yes | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


## Salvando como JPG e aplicando full windowing

In [ ]:
!mkdir small_rsna_ds_as_jpg
!mkdir /content/small_rsna_ds_as_jpg/no
!mkdir /content/small_rsna_ds_as_jpg/yes

!mkdir test_as_jpg
!mkdir test_as_jpg/no
!mkdir test_as_jpg/yes

In [ ]:
def segment_circle(windowed):
    original = windowed.copy().astype("uint8")
    mask = np.zeros(original.shape, dtype=np.uint8)
    gray = original
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)
    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 4 and area > 10000 and area < 500000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            cv2.circle(mask, (int(x), int(y)), int(r), (255, 255, 255), -1)
            cv2.circle(original, (int(x), int(y)), int(r), (36, 255, 12), 0)
    x,y,w,h = cv2.boundingRect(mask)
    mask_ROI = mask[y:y+h, x:x+w]
    image_ROI = original[y:y+h, x:x+w]
    result = cv2.bitwise_and(image_ROI, image_ROI, mask=mask_ROI)
    return result, (x, y, w, h)

def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(img, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img


def preprocess_and_segment(path, make_correct=True):    
    dcm = pydicom.dcmread(path)
    if make_correct and (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    sample = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

    brain_window = window_image(sample, 40, 80)
    dural_window = window_image(sample, 80, 200)
    bone_window = window_image(sample, 600, 2800)

    brain_window, (x,y,w,h) = segment_circle(brain_window)
    if brain_window is None:
        return np.zeros((512, 512, 3))
    dural_window = dural_window[y:y+h, x:x+w]
    bone_window = bone_window[y:y+h, x:x+w]
    
    brain_window = (brain_window - (0.)) / 80.
    dural_window = (dural_window - (-20.)) / 200.
    bone_window = (bone_window - (-1200.)) / 2800.
    img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")

    return img_3ch


def save_W_as_jpg(path, id, dest_folder):

  im_3ch = preprocess_and_segment(path)

  im = window_image(im_3ch, 40, 80)

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

### Para o conjunto Train

In [ ]:
train_path = "/content/small_rsna_ds_train/yes/"
folder = os.listdir("/content/small_rsna_ds_train/yes/")
dest_folder = "/content/small_rsna_ds_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_train/no/"
folder = os.listdir("/content/small_rsna_ds_train/no/")
dest_folder = "/content/small_rsna_ds_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_as_jpg/yes/ | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


### Para o conjunto Test

In [ ]:
train_path = "/content/small_rsna_ds_test/yes/"
folder = os.listdir("/content/small_rsna_ds_test/yes/")
dest_folder = "/content/test_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_test/no/"
folder = os.listdir("/content/small_rsna_ds_test/no/")
dest_folder = "/content/test_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print('Numero de imagens: NO')
!ls /content/test_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/test_as_jpg/yes/ | wc -l

Numero de imagens: NO
10022
Numero de imagens: YES
1643


# Carregando o dataset

In [ ]:
train_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="training",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
     )

Found 23334 files belonging to 2 classes.
Using 15556 files for training.


In [ ]:
val_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="validation",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
)

Found 23334 files belonging to 2 classes.
Using 7778 files for validation.


In [ ]:
test_data = image_dataset_from_directory(
    "/content/test_as_jpg",
     labels='inferred',
     label_mode="int",
     batch_size = 64,
     color_mode = "rgb"
     )

Found 11665 files belonging to 2 classes.


In [ ]:
class_names = train_data.class_names

## Normalizando e Pegando o tamanho de entrada

In [ ]:
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 256, 256, 3)
(64,)


In [ ]:
normalization_layer = Rescaling(1./255)

normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


# Experimentos

## N = 512 [OK]

In [ ]:
N = 512

### L = 1 [OK]

#### D = 0 [OK]

In [ ]:
NAME = "N512-L1-D0"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 75s 246ms/step - loss: 1.4177 - auc: 0.6011 - accuracy: 0.8145 - val_loss: 0.8785 - val_auc: 0.7520 - val_accuracy: 0.6305
Epoch 2/100
244/244 [==============================] - 60s 241ms/step - loss: 0.4418 - auc: 0.7290 - accuracy: 0.8429 - val_loss: 0.4296 - val_auc: 0.7387 - val_accuracy: 0.8631
Epoch 3/100
244/244 [==============================] - 64s 259ms/step - loss: 0.3736 - auc: 0.7848 - accuracy: 0.8594 - val_loss: 0.4378 - val_auc: 0.7826 - val_accuracy: 0.7948
Epoch 4/100
244/244 [==============================] - 60s 243ms/step - loss: 0.3432 - auc: 0.8108 - accuracy: 0.8687 - val_loss: 0.7476 - val_auc: 0.7907 - val_accuracy: 0.6520
Epoch 5/100
244/244 [==============================] - 61s 246ms/step - loss: 0.3459 - auc: 0.8182 - accuracy: 0.8707 - val_loss: 0.3974 - val_auc: 0.7982 - val_accuracy: 0.8277
Epoch 6/100
244/244 [==============================] - 60s 245ms/step - loss: 0.2813 - auc: 0.8645 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 22s 112ms/step - loss: 0.6986 - auc: 0.7940 - accuracy: 0.6905
[0.698626697063446, 0.7940026521682739, 0.6905272006988525]


AttributeError: ignored

In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 1 [OK]

In [ ]:
NAME = "N512-L1-D1"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 77s 307ms/step - loss: 1.0723 - auc: 0.5539 - accuracy: 0.8289 - val_loss: 0.3900 - val_auc: 0.7488 - val_accuracy: 0.8579
Epoch 2/100
244/244 [==============================] - 72s 295ms/step - loss: 0.3857 - auc: 0.6899 - accuracy: 0.8573 - val_loss: 0.3594 - val_auc: 0.7613 - val_accuracy: 0.8582
Epoch 3/100
244/244 [==============================] - 74s 299ms/step - loss: 0.3758 - auc: 0.7150 - accuracy: 0.8580 - val_loss: 0.3778 - val_auc: 0.7726 - val_accuracy: 0.8583
Epoch 4/100
244/244 [==============================] - 70s 284ms/step - loss: 0.3688 - auc: 0.7266 - accuracy: 0.8588 - val_loss: 0.3680 - val_auc: 0.7769 - val_accuracy: 0.8583
Epoch 5/100
244/244 [==============================] - 77s 314ms/step - loss: 0.3647 - auc: 0.7345 - accuracy: 0.8590 - val_loss: 0.3530 - val_auc: 0.7838 - val_accuracy: 0.8582
Epoch 6/100
244/244 [==============================] - 70s 283ms/step - loss: 0.3602 - auc: 0.7464 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 113ms/step - loss: 0.3355 - auc: 0.7972 - accuracy: 0.8660
[0.3355242609977722, 0.7971541881561279, 0.8660094141960144]


AttributeError: ignored

In [ ]:
import pickle
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 2 [OK]

In [ ]:
NAME = "N512-L1-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 62s 246ms/step - loss: 1.3198 - auc: 0.5556 - accuracy: 0.8293 - val_loss: 0.3654 - val_auc: 0.7477 - val_accuracy: 0.8582
Epoch 2/100
244/244 [==============================] - 61s 246ms/step - loss: 0.3722 - auc: 0.7191 - accuracy: 0.8583 - val_loss: 0.3518 - val_auc: 0.7653 - val_accuracy: 0.8583
Epoch 3/100
244/244 [==============================] - 61s 248ms/step - loss: 0.3616 - auc: 0.7415 - accuracy: 0.8588 - val_loss: 0.3529 - val_auc: 0.7826 - val_accuracy: 0.8592
Epoch 4/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3543 - auc: 0.7560 - accuracy: 0.8608 - val_loss: 0.3456 - val_auc: 0.7805 - val_accuracy: 0.8595
Epoch 5/100
244/244 [==============================] - 60s 241ms/step - loss: 0.3505 - auc: 0.7635 - accuracy: 0.8618 - val_loss: 0.3401 - val_auc: 0.7938 - val_accuracy: 0.8587
Epoch 6/100
244/244 [==============================] - 60s 242ms/step - loss: 0.3472 - auc: 0.7704 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

183/183 [==============================] - 20s 104ms/step - loss: 0.3285 - auc: 0.8093 - accuracy: 0.8701
[0.3284884989261627, 0.8093360066413879, 0.8701242804527283]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 3 [OK]

In [ ]:
NAME = "N512-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 72s 231ms/step - loss: 0.8522 - auc: 0.5784 - accuracy: 0.8348 - val_loss: 0.3597 - val_auc: 0.7525 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 56s 227ms/step - loss: 0.3672 - auc: 0.7314 - accuracy: 0.8581 - val_loss: 0.3456 - val_auc: 0.7817 - val_accuracy: 0.8582
Epoch 3/100
244/244 [==============================] - 56s 229ms/step - loss: 0.3575 - auc: 0.7502 - accuracy: 0.8595 - val_loss: 0.3536 - val_auc: 0.7858 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 231ms/step - loss: 0.3539 - auc: 0.7598 - accuracy: 0.8604 - val_loss: 0.3367 - val_auc: 0.7982 - val_accuracy: 0.8623
Epoch 5/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3445 - auc: 0.7741 - accuracy: 0.8637 - val_loss: 0.3342 - val_auc: 0.8005 - val_accuracy: 0.8606
Epoch 6/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3388 - auc: 0.7835 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

183/183 [==============================] - 21s 110ms/step - loss: 0.3264 - auc: 0.8136 - accuracy: 0.8753
[0.3264344036579132, 0.8135601878166199, 0.8752679228782654]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 4 [OK]

In [ ]:
NAME = "N512-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 60s 237ms/step - loss: 1.0176 - auc: 0.5830 - accuracy: 0.8388 - val_loss: 0.3538 - val_auc: 0.7652 - val_accuracy: 0.8575
Epoch 2/100
244/244 [==============================] - 57s 233ms/step - loss: 0.3661 - auc: 0.7323 - accuracy: 0.8592 - val_loss: 0.3459 - val_auc: 0.7866 - val_accuracy: 0.8579
Epoch 3/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3589 - auc: 0.7452 - accuracy: 0.8579 - val_loss: 0.3446 - val_auc: 0.7878 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3518 - auc: 0.7622 - accuracy: 0.8596 - val_loss: 0.3421 - val_auc: 0.7889 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 57s 229ms/step - loss: 0.3429 - auc: 0.7788 - accuracy: 0.8613 - val_loss: 0.3587 - val_auc: 0.7851 - val_accuracy: 0.8582
Epoch 6/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3436 - auc: 0.7767 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 18s 97ms/step - loss: 0.3246 - auc: 0.8087 - accuracy: 0.8731
[0.32456985116004944, 0.808731198310852, 0.8731247186660767]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 5 [OK]

In [ ]:
NAME = "N512-L1-D5"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 68s 274ms/step - loss: 2.1043 - auc: 0.5519 - accuracy: 0.8010 - val_loss: 0.4299 - val_auc: 0.6693 - val_accuracy: 0.8547
Epoch 2/100
244/244 [==============================] - 68s 278ms/step - loss: 0.5557 - auc: 0.6376 - accuracy: 0.8192 - val_loss: 0.4495 - val_auc: 0.6752 - val_accuracy: 0.8586
Epoch 3/100
244/244 [==============================] - 67s 273ms/step - loss: 0.5246 - auc: 0.6666 - accuracy: 0.8208 - val_loss: 0.3840 - val_auc: 0.7385 - val_accuracy: 0.8345
Epoch 4/100
244/244 [==============================] - 67s 272ms/step - loss: 0.4862 - auc: 0.6834 - accuracy: 0.8264 - val_loss: 0.3602 - val_auc: 0.7554 - val_accuracy: 0.8601
Epoch 5/100
244/244 [==============================] - 67s 274ms/step - loss: 0.4258 - auc: 0.7191 - accuracy: 0.8363 - val_loss: 0.3528 - val_auc: 0.7791 - val_accuracy: 0.8560
Epoch 6/100
244/244 [==============================] - 67s 274ms/step - loss: 0.4159 - auc: 0.7188 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 18s 98ms/step - loss: 0.3408 - auc: 0.7831 - accuracy: 0.8657
[0.340798020362854, 0.783123254776001, 0.8656665086746216]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 6 [OK]

In [ ]:
NAME = "N512-L1-D6"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 58s 234ms/step - loss: 1.3578 - auc: 0.5996 - accuracy: 0.8213 - val_loss: 0.3713 - val_auc: 0.7438 - val_accuracy: 0.8494
Epoch 2/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3894 - auc: 0.7362 - accuracy: 0.8442 - val_loss: 0.6298 - val_auc: 0.6400 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3702 - auc: 0.7624 - accuracy: 0.8544 - val_loss: 0.4579 - val_auc: 0.7168 - val_accuracy: 0.8590
Epoch 4/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3184 - auc: 0.8185 - accuracy: 0.8721 - val_loss: 0.3933 - val_auc: 0.7591 - val_accuracy: 0.8631
Epoch 5/100
244/244 [==============================] - 58s 236ms/step - loss: 0.2962 - auc: 0.8448 - accuracy: 0.8811 - val_loss: 0.3786 - val_auc: 0.7644 - val_accuracy: 0.8660
Epoch 6/100
244/244 [==============================] - 67s 272ms/step - loss: 0.2784 - auc: 0.8604 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.4957 - auc: 0.8080 - accuracy: 0.7697
[0.49569642543792725, 0.8080277442932129, 0.769652783870697]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

### L = 2 [OK]

#### D = 0 [OK]

In [ ]:
NAME = "N512-L2-D0"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 60s 240ms/step - loss: 0.5729 - auc: 0.6147 - accuracy: 0.8372 - val_loss: 0.3576 - val_auc: 0.7499 - val_accuracy: 0.8596
Epoch 2/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3563 - auc: 0.7628 - accuracy: 0.8572 - val_loss: 0.3469 - val_auc: 0.7746 - val_accuracy: 0.8570
Epoch 3/100
244/244 [==============================] - 57s 231ms/step - loss: 0.3244 - auc: 0.8056 - accuracy: 0.8712 - val_loss: 0.3981 - val_auc: 0.7853 - val_accuracy: 0.8186
Epoch 4/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3229 - auc: 0.8092 - accuracy: 0.8716 - val_loss: 0.3849 - val_auc: 0.7740 - val_accuracy: 0.8629
Epoch 5/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3023 - auc: 0.8346 - accuracy: 0.8804 - val_loss: 0.3531 - val_auc: 0.7857 - val_accuracy: 0.8636
Epoch 6/100
244/244 [==============================] - 58s 234ms/step - loss: 0.2875 - auc: 0.8520 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.4127 - auc: 0.7963 - accuracy: 0.8076
[0.41273799538612366, 0.7962908148765564, 0.8076296448707581]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 1 [OK]

In [ ]:
NAME = "N512-L2-D1"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 72s 274ms/step - loss: 1.4529 - auc: 0.5321 - accuracy: 0.7981 - val_loss: 0.3998 - val_auc: 0.6970 - val_accuracy: 0.8579
Epoch 2/100
244/244 [==============================] - 66s 268ms/step - loss: 0.4917 - auc: 0.6268 - accuracy: 0.8325 - val_loss: 0.3700 - val_auc: 0.7329 - val_accuracy: 0.8578
Epoch 3/100
244/244 [==============================] - 67s 273ms/step - loss: 0.4847 - auc: 0.6329 - accuracy: 0.8366 - val_loss: 0.3850 - val_auc: 0.7338 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 66s 268ms/step - loss: 0.4672 - auc: 0.6584 - accuracy: 0.8485 - val_loss: 0.3847 - val_auc: 0.7409 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 67s 272ms/step - loss: 0.4733 - auc: 0.6369 - accuracy: 0.8507 - val_loss: 0.3553 - val_auc: 0.7566 - val_accuracy: 0.8581
Epoch 7/100
244/244 [==============================] - 68s 276ms/step - loss: 0.4826 - auc: 0.6244 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 23s 117ms/step - loss: 0.3531 - auc: 0.7610 - accuracy: 0.8592
[0.3531126081943512, 0.760989248752594, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 2 [OK]

In [ ]:
NAME = "N512-L2-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 61s 242ms/step - loss: 0.8784 - auc: 0.5460 - accuracy: 0.8124 - val_loss: 0.3817 - val_auc: 0.7399 - val_accuracy: 0.8565
Epoch 2/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3872 - auc: 0.7004 - accuracy: 0.8547 - val_loss: 0.3550 - val_auc: 0.7640 - val_accuracy: 0.8578
Epoch 3/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3670 - auc: 0.7433 - accuracy: 0.8545 - val_loss: 0.3534 - val_auc: 0.7774 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 58s 236ms/step - loss: 0.3554 - auc: 0.7596 - accuracy: 0.8584 - val_loss: 0.3548 - val_auc: 0.7917 - val_accuracy: 0.8588
Epoch 5/100
244/244 [==============================] - 58s 236ms/step - loss: 0.3864 - auc: 0.7082 - accuracy: 0.8540 - val_loss: 0.3658 - val_auc: 0.7736 - val_accuracy: 0.8583
Epoch 6/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3580 - auc: 0.7540 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 101ms/step - loss: 0.3370 - auc: 0.8059 - accuracy: 0.8592
[0.3370477855205536, 0.805869996547699, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 3 [OK]

In [ ]:
NAME = "N512-L2-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 58s 233ms/step - loss: 0.8075 - auc: 0.5481 - accuracy: 0.8177 - val_loss: 0.3998 - val_auc: 0.7314 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 57s 233ms/step - loss: 0.4472 - auc: 0.6494 - accuracy: 0.8372 - val_loss: 0.3795 - val_auc: 0.7374 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 57s 231ms/step - loss: 0.4299 - auc: 0.6671 - accuracy: 0.8477 - val_loss: 0.3963 - val_auc: 0.7585 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 233ms/step - loss: 0.4224 - auc: 0.6888 - accuracy: 0.8511 - val_loss: 0.3523 - val_auc: 0.7690 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 57s 230ms/step - loss: 0.4141 - auc: 0.6929 - accuracy: 0.8530 - val_loss: 0.3665 - val_auc: 0.7758 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3937 - auc: 0.7232 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3428 - auc: 0.7982 - accuracy: 0.8592
[0.34282225370407104, 0.7981623411178589, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 4 [OK]

In [ ]:
NAME = "N512-L2-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 57s 225ms/step - loss: 0.9068 - auc: 0.5542 - accuracy: 0.8128 - val_loss: 0.3710 - val_auc: 0.7420 - val_accuracy: 0.8572
Epoch 2/100
244/244 [==============================] - 56s 227ms/step - loss: 0.3786 - auc: 0.7177 - accuracy: 0.8520 - val_loss: 0.3561 - val_auc: 0.7651 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3599 - auc: 0.7485 - accuracy: 0.8576 - val_loss: 0.3457 - val_auc: 0.7782 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3597 - auc: 0.7501 - accuracy: 0.8570 - val_loss: 0.3479 - val_auc: 0.7775 - val_accuracy: 0.8600
Epoch 5/100
244/244 [==============================] - 56s 229ms/step - loss: 0.3443 - auc: 0.7767 - accuracy: 0.8602 - val_loss: 0.3401 - val_auc: 0.7902 - val_accuracy: 0.8583
Epoch 6/100
244/244 [==============================] - 57s 231ms/step - loss: 0.3348 - auc: 0.7932 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3339 - auc: 0.7976 - accuracy: 0.8690
[0.33392134308815, 0.7975985407829285, 0.8690098524093628]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 5 [OK]

In [ ]:
NAME = "N512-L2-D5"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 68s 273ms/step - loss: 0.6799 - auc: 0.5931 - accuracy: 0.8266 - val_loss: 0.6012 - val_auc: 0.7490 - val_accuracy: 0.6481
Epoch 2/100
244/244 [==============================] - 67s 272ms/step - loss: 0.4034 - auc: 0.7053 - accuracy: 0.8411 - val_loss: 0.6863 - val_auc: 0.7307 - val_accuracy: 0.5582
Epoch 3/100
244/244 [==============================] - 67s 273ms/step - loss: 0.3805 - auc: 0.7361 - accuracy: 0.8463 - val_loss: 0.5560 - val_auc: 0.7654 - val_accuracy: 0.6758
Epoch 4/100
244/244 [==============================] - 68s 278ms/step - loss: 0.3656 - auc: 0.7526 - accuracy: 0.8527 - val_loss: 0.9051 - val_auc: 0.7468 - val_accuracy: 0.4317
Epoch 5/100
244/244 [==============================] - 69s 282ms/step - loss: 0.3832 - auc: 0.7335 - accuracy: 0.8457 - val_loss: 0.4066 - val_auc: 0.7654 - val_accuracy: 0.8394
Epoch 6/100
244/244 [==============================] - 68s 276ms/step - loss: 0.3419 - auc: 0.7806 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3558 - auc: 0.7945 - accuracy: 0.8588
[0.35584476590156555, 0.7945382595062256, 0.8588083982467651]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 6 [OK]

In [ ]:
NAME = "N512-L2-D6"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 58s 230ms/step - loss: 0.6932 - auc: 0.6147 - accuracy: 0.8302 - val_loss: 0.4832 - val_auc: 0.7743 - val_accuracy: 0.7483
Epoch 2/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3663 - auc: 0.7511 - accuracy: 0.8563 - val_loss: 0.3514 - val_auc: 0.7723 - val_accuracy: 0.8629
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3358 - auc: 0.7918 - accuracy: 0.8649 - val_loss: 0.3679 - val_auc: 0.7896 - val_accuracy: 0.8426
Epoch 4/100
244/244 [==============================] - 57s 233ms/step - loss: 0.3180 - auc: 0.8147 - accuracy: 0.8736 - val_loss: 0.4065 - val_auc: 0.7421 - val_accuracy: 0.8600
Epoch 5/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3057 - auc: 0.8327 - accuracy: 0.8786 - val_loss: 0.3948 - val_auc: 0.7989 - val_accuracy: 0.8194
Epoch 6/100
244/244 [==============================] - 59s 240ms/step - loss: 0.2846 - auc: 0.8545 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3342 - auc: 0.7975 - accuracy: 0.8658
[0.334225594997406, 0.7974523305892944, 0.8658379912376404]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

### L = 3 [OK]

#### D = 0 [OK]

In [ ]:
NAME = "N512-L3-D0"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 82s 271ms/step - loss: 0.4950 - auc: 0.6255 - accuracy: 0.8388 - val_loss: 0.3647 - val_auc: 0.7815 - val_accuracy: 0.8547
Epoch 2/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3629 - auc: 0.7516 - accuracy: 0.8538 - val_loss: 0.4011 - val_auc: 0.7598 - val_accuracy: 0.8590
Epoch 3/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3329 - auc: 0.7943 - accuracy: 0.8671 - val_loss: 0.3958 - val_auc: 0.8017 - val_accuracy: 0.8101
Epoch 4/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3126 - auc: 0.8211 - accuracy: 0.8768 - val_loss: 0.3360 - val_auc: 0.7933 - val_accuracy: 0.8687
Epoch 5/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3043 - auc: 0.8324 - accuracy: 0.8788 - val_loss: 0.4370 - val_auc: 0.8063 - val_accuracy: 0.7816
Epoch 6/100
244/244 [==============================] - 58s 234ms/step - loss: 0.2926 - auc: 0.8491 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 112ms/step - loss: 0.3268 - auc: 0.8064 - accuracy: 0.8709
[0.32677069306373596, 0.8063855171203613, 0.8708958625793457]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 1 [OK]

In [ ]:
NAME = "N512-L3-D1"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 73s 285ms/step - loss: 1.2899 - auc: 0.5270 - accuracy: 0.7536 - val_loss: 0.4853 - val_auc: 0.6803 - val_accuracy: 0.8572
Epoch 2/100
244/244 [==============================] - 69s 281ms/step - loss: 0.5127 - auc: 0.5970 - accuracy: 0.8058 - val_loss: 0.3840 - val_auc: 0.7371 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 68s 277ms/step - loss: 0.4625 - auc: 0.6441 - accuracy: 0.8381 - val_loss: 0.3976 - val_auc: 0.7437 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 68s 276ms/step - loss: 0.4470 - auc: 0.6571 - accuracy: 0.8468 - val_loss: 0.3746 - val_auc: 0.7522 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 68s 276ms/step - loss: 0.4334 - auc: 0.6735 - accuracy: 0.8501 - val_loss: 0.3636 - val_auc: 0.7686 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 68s 277ms/step - loss: 0.4182 - auc: 0.6884 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3557 - auc: 0.7686 - accuracy: 0.8592
[0.35566288232803345, 0.7686297297477722, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 2 [OK]

In [ ]:
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 61s 243ms/step - loss: 0.5247 - auc: 0.5750 - accuracy: 0.8338 - val_loss: 0.3768 - val_auc: 0.6819 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 59s 239ms/step - loss: 0.3694 - auc: 0.7259 - accuracy: 0.8568 - val_loss: 0.3423 - val_auc: 0.7890 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3455 - auc: 0.7719 - accuracy: 0.8614 - val_loss: 0.3421 - val_auc: 0.7911 - val_accuracy: 0.8583
Epoch 4/100
244/244 [==============================] - 59s 237ms/step - loss: 0.3344 - auc: 0.7917 - accuracy: 0.8646 - val_loss: 0.3358 - val_auc: 0.8039 - val_accuracy: 0.8669
Epoch 5/100
244/244 [==============================] - 59s 238ms/step - loss: 0.3200 - auc: 0.8150 - accuracy: 0.8685 - val_loss: 0.3267 - val_auc: 0.8113 - val_accuracy: 0.8650
Epoch 6/100
244/244 [==============================] - 59s 239ms/step - loss: 0.3018 - auc: 0.8401 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3521 - auc: 0.8189 - accuracy: 0.8541
[0.35206112265586853, 0.8188831210136414, 0.8540934324264526]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 3 [OK]

In [ ]:
NAME = "N512-L3-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 60s 236ms/step - loss: 0.9955 - auc: 0.5359 - accuracy: 0.8093 - val_loss: 0.4081 - val_auc: 0.6047 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 59s 239ms/step - loss: 0.4732 - auc: 0.6126 - accuracy: 0.8352 - val_loss: 0.3691 - val_auc: 0.7494 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 58s 236ms/step - loss: 0.4401 - auc: 0.6620 - accuracy: 0.8480 - val_loss: 0.3574 - val_auc: 0.7579 - val_accuracy: 0.8584
Epoch 4/100
244/244 [==============================] - 58s 236ms/step - loss: 0.4225 - auc: 0.6808 - accuracy: 0.8540 - val_loss: 0.3731 - val_auc: 0.7602 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 59s 238ms/step - loss: 0.4244 - auc: 0.6834 - accuracy: 0.8552 - val_loss: 0.3516 - val_auc: 0.7687 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 58s 234ms/step - loss: 0.4010 - auc: 0.7096 - accuracy:

In [ ]:
results = model.evaluate(test_data) 
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3389 - auc: 0.7918 - accuracy: 0.8592
[0.3389175236225128, 0.7917983531951904, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 4 [OK]

In [ ]:
NAME = "N512-L3-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 59s 237ms/step - loss: 0.7742 - auc: 0.5549 - accuracy: 0.8149 - val_loss: 0.3994 - val_auc: 0.6433 - val_accuracy: 0.8578
Epoch 2/100
244/244 [==============================] - 58s 236ms/step - loss: 0.3861 - auc: 0.7002 - accuracy: 0.8547 - val_loss: 0.3549 - val_auc: 0.7683 - val_accuracy: 0.8582
Epoch 3/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3617 - auc: 0.7451 - accuracy: 0.8573 - val_loss: 0.3449 - val_auc: 0.7791 - val_accuracy: 0.8586
Epoch 4/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3490 - auc: 0.7681 - accuracy: 0.8586 - val_loss: 0.3507 - val_auc: 0.7832 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3488 - auc: 0.7708 - accuracy: 0.8582 - val_loss: 0.3400 - val_auc: 0.7883 - val_accuracy: 0.8582
Epoch 6/100
244/244 [==============================] - 58s 236ms/step - loss: 0.3302 - auc: 0.8013 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3540 - auc: 0.7903 - accuracy: 0.8509
[0.3539787530899048, 0.7902542948722839, 0.8509215712547302]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 5 [OK]

In [ ]:
NAME = "N512-L3-D5"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 71s 283ms/step - loss: 0.5374 - auc: 0.5902 - accuracy: 0.8353 - val_loss: 0.3915 - val_auc: 0.7245 - val_accuracy: 0.8537
Epoch 2/100
244/244 [==============================] - 77s 313ms/step - loss: 0.3775 - auc: 0.7239 - accuracy: 0.8516 - val_loss: 0.4069 - val_auc: 0.7632 - val_accuracy: 0.8467
Epoch 3/100
244/244 [==============================] - 70s 284ms/step - loss: 0.3611 - auc: 0.7490 - accuracy: 0.8559 - val_loss: 0.3527 - val_auc: 0.7763 - val_accuracy: 0.8584
Epoch 4/100
244/244 [==============================] - 69s 279ms/step - loss: 0.3558 - auc: 0.7571 - accuracy: 0.8579 - val_loss: 0.4441 - val_auc: 0.7799 - val_accuracy: 0.8176
Epoch 5/100
244/244 [==============================] - 69s 279ms/step - loss: 0.3463 - auc: 0.7736 - accuracy: 0.8599 - val_loss: 0.5387 - val_auc: 0.7614 - val_accuracy: 0.7070
Epoch 6/100
244/244 [==============================] - 70s 283ms/step - loss: 0.3475 - auc: 0.7721 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3408 - auc: 0.7942 - accuracy: 0.8668
[0.34078672528266907, 0.7941610217094421, 0.8667809963226318]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 6 [OK]

In [ ]:
NAME = "N512-L3-D6"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 59s 237ms/step - loss: 0.4872 - auc: 0.6129 - accuracy: 0.8395 - val_loss: 0.4423 - val_auc: 0.7392 - val_accuracy: 0.8179
Epoch 2/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3577 - auc: 0.7537 - accuracy: 0.8600 - val_loss: 0.3466 - val_auc: 0.7936 - val_accuracy: 0.8611
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3291 - auc: 0.7976 - accuracy: 0.8692 - val_loss: 0.3363 - val_auc: 0.8065 - val_accuracy: 0.8642
Epoch 4/100
244/244 [==============================] - 59s 238ms/step - loss: 0.3078 - auc: 0.8259 - accuracy: 0.8783 - val_loss: 0.3703 - val_auc: 0.7837 - val_accuracy: 0.8622
Epoch 5/100
244/244 [==============================] - 59s 239ms/step - loss: 0.2884 - auc: 0.8509 - accuracy: 0.8878 - val_loss: 0.3901 - val_auc: 0.8096 - val_accuracy: 0.8221
Epoch 6/100
244/244 [==============================] - 59s 240ms/step - loss: 0.2690 - auc: 0.8695 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 112ms/step - loss: 0.3347 - auc: 0.8046 - accuracy: 0.8633
[0.3347075581550598, 0.8046112656593323, 0.8632661700248718]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

## N = 1024

In [ ]:
N = 1024

### L = 1

#### D = 0 [OK]

In [ ]:
NAME = "N1024-L1-D0"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 90s 303ms/step - loss: 1.6937 - auc: 0.6041 - accuracy: 0.8232 - val_loss: 0.4682 - val_auc: 0.6823 - val_accuracy: 0.8587
Epoch 2/100
244/244 [==============================] - 65s 261ms/step - loss: 0.4376 - auc: 0.7126 - accuracy: 0.8401 - val_loss: 0.8691 - val_auc: 0.5450 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 71s 288ms/step - loss: 0.3678 - auc: 0.7680 - accuracy: 0.8602 - val_loss: 0.4032 - val_auc: 0.7847 - val_accuracy: 0.8002
Epoch 4/100
244/244 [==============================] - 62s 253ms/step - loss: 0.3305 - auc: 0.8071 - accuracy: 0.8715 - val_loss: 0.3458 - val_auc: 0.7884 - val_accuracy: 0.8633
Epoch 5/100
244/244 [==============================] - 62s 252ms/step - loss: 0.3289 - auc: 0.8181 - accuracy: 0.8741 - val_loss: 0.3831 - val_auc: 0.7715 - val_accuracy: 0.8671
Epoch 6/100
244/244 [==============================] - 62s 252ms/step - loss: 0.2934 - auc: 0.8478 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 22s 112ms/step - loss: 0.5315 - auc: 0.7882 - accuracy: 0.7438
[0.5315011143684387, 0.7882260084152222, 0.7437633872032166]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 1 [OK]

In [ ]:
NAME = "N1024-L1-D1"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 74s 298ms/step - loss: 1.7129 - auc: 0.5343 - accuracy: 0.8142 - val_loss: 0.3898 - val_auc: 0.7398 - val_accuracy: 0.8579
Epoch 2/100
244/244 [==============================] - 73s 296ms/step - loss: 0.3875 - auc: 0.6840 - accuracy: 0.8578 - val_loss: 0.3602 - val_auc: 0.7640 - val_accuracy: 0.8582
Epoch 3/100
244/244 [==============================] - 72s 293ms/step - loss: 0.3824 - auc: 0.6927 - accuracy: 0.8575 - val_loss: 0.3668 - val_auc: 0.7643 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 72s 294ms/step - loss: 0.3743 - auc: 0.7074 - accuracy: 0.8597 - val_loss: 0.3570 - val_auc: 0.7685 - val_accuracy: 0.8583
Epoch 5/100
244/244 [==============================] - 72s 294ms/step - loss: 0.3683 - auc: 0.7235 - accuracy: 0.8591 - val_loss: 0.3565 - val_auc: 0.7717 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 73s 295ms/step - loss: 0.3680 - auc: 0.7262 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3382 - auc: 0.7838 - accuracy: 0.8643
[0.3381996750831604, 0.7838283777236938, 0.8642948865890503]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 2 [OK]

In [ ]:
NAME = "N1024-L1-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 81s 273ms/step - loss: 1.8306 - auc: 0.5617 - accuracy: 0.8224 - val_loss: 0.3718 - val_auc: 0.7270 - val_accuracy: 0.8586
Epoch 2/100
244/244 [==============================] - 63s 256ms/step - loss: 0.3787 - auc: 0.7113 - accuracy: 0.8549 - val_loss: 0.3542 - val_auc: 0.7686 - val_accuracy: 0.8591
Epoch 3/100
244/244 [==============================] - 63s 257ms/step - loss: 0.3691 - auc: 0.7276 - accuracy: 0.8583 - val_loss: 0.3574 - val_auc: 0.7661 - val_accuracy: 0.8584
Epoch 4/100
244/244 [==============================] - 65s 262ms/step - loss: 0.3590 - auc: 0.7481 - accuracy: 0.8592 - val_loss: 0.3464 - val_auc: 0.7878 - val_accuracy: 0.8593
Epoch 5/100
244/244 [==============================] - 62s 253ms/step - loss: 0.3576 - auc: 0.7498 - accuracy: 0.8593 - val_loss: 0.3438 - val_auc: 0.7873 - val_accuracy: 0.8587
Epoch 6/100
244/244 [==============================] - 63s 257ms/step - loss: 0.3548 - auc: 0.7560 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

183/183 [==============================] - 24s 127ms/step - loss: 0.3282 - auc: 0.8076 - accuracy: 0.8721
[0.32820528745651245, 0.8075695037841797, 0.8720960021018982]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 3 [OK]

In [ ]:
NAME = "N1024-L1-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 65s 261ms/step - loss: 1.1366 - auc: 0.5781 - accuracy: 0.8262 - val_loss: 0.3540 - val_auc: 0.7648 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 64s 259ms/step - loss: 0.3668 - auc: 0.7343 - accuracy: 0.8583 - val_loss: 0.3491 - val_auc: 0.7857 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 62s 251ms/step - loss: 0.3563 - auc: 0.7541 - accuracy: 0.8606 - val_loss: 0.3404 - val_auc: 0.7935 - val_accuracy: 0.8628
Epoch 4/100
244/244 [==============================] - 63s 256ms/step - loss: 0.3517 - auc: 0.7643 - accuracy: 0.8600 - val_loss: 0.3371 - val_auc: 0.7965 - val_accuracy: 0.8595
Epoch 5/100
244/244 [==============================] - 62s 253ms/step - loss: 0.3465 - auc: 0.7715 - accuracy: 0.8622 - val_loss: 0.3395 - val_auc: 0.7945 - val_accuracy: 0.8593
Epoch 6/100
244/244 [==============================] - 63s 255ms/step - loss: 0.3343 - auc: 0.7902 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

183/183 [==============================] - 21s 109ms/step - loss: 0.3286 - auc: 0.8138 - accuracy: 0.8754
[0.32860010862350464, 0.8137543201446533, 0.8753536343574524]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 4 [OK]

In [ ]:
NAME = "N1024-L1-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 66s 265ms/step - loss: 1.4062 - auc: 0.5738 - accuracy: 0.8273 - val_loss: 0.3553 - val_auc: 0.7627 - val_accuracy: 0.8582
Epoch 2/100
244/244 [==============================] - 65s 263ms/step - loss: 0.3674 - auc: 0.7311 - accuracy: 0.8561 - val_loss: 0.3485 - val_auc: 0.7760 - val_accuracy: 0.8595
Epoch 3/100
244/244 [==============================] - 63s 257ms/step - loss: 0.3583 - auc: 0.7488 - accuracy: 0.8584 - val_loss: 0.3723 - val_auc: 0.7755 - val_accuracy: 0.8583
Epoch 4/100
244/244 [==============================] - 64s 259ms/step - loss: 0.3501 - auc: 0.7627 - accuracy: 0.8619 - val_loss: 0.4121 - val_auc: 0.7592 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 63s 256ms/step - loss: 0.3448 - auc: 0.7738 - accuracy: 0.8628 - val_loss: 0.3390 - val_auc: 0.7934 - val_accuracy: 0.8614
Epoch 6/100
244/244 [==============================] - 64s 258ms/step - loss: 0.3384 - auc: 0.7856 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 109ms/step - loss: 0.3381 - auc: 0.8200 - accuracy: 0.8643
[0.33814737200737, 0.8199527859687805, 0.8642948865890503]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 5 [OK]

In [ ]:
NAME = "N1024-L1-D5"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 76s 304ms/step - loss: 1.2912 - auc: 0.5901 - accuracy: 0.8159 - val_loss: 0.3673 - val_auc: 0.7529 - val_accuracy: 0.8520
Epoch 2/100
244/244 [==============================] - 73s 295ms/step - loss: 0.4268 - auc: 0.7008 - accuracy: 0.8360 - val_loss: 0.3441 - val_auc: 0.7770 - val_accuracy: 0.8619
Epoch 3/100
244/244 [==============================] - 73s 297ms/step - loss: 0.3848 - auc: 0.7413 - accuracy: 0.8482 - val_loss: 0.3803 - val_auc: 0.7578 - val_accuracy: 0.8588
Epoch 4/100
244/244 [==============================] - 73s 298ms/step - loss: 0.3747 - auc: 0.7540 - accuracy: 0.8530 - val_loss: 0.3987 - val_auc: 0.7889 - val_accuracy: 0.8079
Epoch 5/100
244/244 [==============================] - 72s 292ms/step - loss: 0.3518 - auc: 0.7783 - accuracy: 0.8577 - val_loss: 0.3416 - val_auc: 0.7857 - val_accuracy: 0.8635
Epoch 6/100
244/244 [==============================] - 72s 294ms/step - loss: 0.3416 - auc: 0.7893 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 20s 108ms/step - loss: 0.3540 - auc: 0.7959 - accuracy: 0.8684
[0.35395851731300354, 0.7959303855895996, 0.8684097528457642]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 6 [OK]

In [ ]:
NAME = "N1024-L1-D6"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 68s 272ms/step - loss: 3.5517 - auc: 0.5621 - accuracy: 0.8030 - val_loss: 1.9762 - val_auc: 0.7414 - val_accuracy: 0.5941
Epoch 2/100
244/244 [==============================] - 63s 256ms/step - loss: 0.9554 - auc: 0.6394 - accuracy: 0.8187 - val_loss: 0.6583 - val_auc: 0.6561 - val_accuracy: 0.8572
Epoch 3/100
244/244 [==============================] - 63s 255ms/step - loss: 0.7589 - auc: 0.6785 - accuracy: 0.8260 - val_loss: 0.7397 - val_auc: 0.6293 - val_accuracy: 0.8610
Epoch 4/100
244/244 [==============================] - 63s 256ms/step - loss: 0.7907 - auc: 0.6745 - accuracy: 0.8291 - val_loss: 0.5008 - val_auc: 0.7541 - val_accuracy: 0.8295
Epoch 5/100
244/244 [==============================] - 63s 257ms/step - loss: 0.4908 - auc: 0.7614 - accuracy: 0.8548 - val_loss: 0.4178 - val_auc: 0.7514 - val_accuracy: 0.8557
Epoch 6/100
244/244 [==============================] - 64s 258ms/step - loss: 0.4309 - auc: 0.7878 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.5416 - auc: 0.7842 - accuracy: 0.7649
[0.5416334271430969, 0.7842249274253845, 0.7648521065711975]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

### L = 2

#### D = 0 [OK]

In [ ]:
NAME = "N1024-L2-D0"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 63s 253ms/step - loss: 0.7705 - auc: 0.6082 - accuracy: 0.8244 - val_loss: 0.6505 - val_auc: 0.6284 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 61s 249ms/step - loss: 0.3927 - auc: 0.7372 - accuracy: 0.8481 - val_loss: 0.4246 - val_auc: 0.7842 - val_accuracy: 0.7858
Epoch 3/100
244/244 [==============================] - 62s 252ms/step - loss: 0.3423 - auc: 0.7949 - accuracy: 0.8644 - val_loss: 0.5177 - val_auc: 0.7964 - val_accuracy: 0.7181
Epoch 4/100
244/244 [==============================] - 62s 251ms/step - loss: 0.3223 - auc: 0.8129 - accuracy: 0.8722 - val_loss: 0.3511 - val_auc: 0.7956 - val_accuracy: 0.8709
Epoch 5/100
244/244 [==============================] - 62s 252ms/step - loss: 0.2819 - auc: 0.8573 - accuracy: 0.8896 - val_loss: 0.3395 - val_auc: 0.7926 - val_accuracy: 0.8631
Epoch 6/100
244/244 [==============================] - 62s 253ms/step - loss: 0.2656 - auc: 0.8728 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.7409 - auc: 0.8071 - accuracy: 0.6772
[0.7409282326698303, 0.8070726990699768, 0.6772395968437195]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 1 [OK]

In [ ]:
NAME = "N1024-L2-D1"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 73s 294ms/step - loss: 1.7892 - auc: 0.5170 - accuracy: 0.8100 - val_loss: 0.4066 - val_auc: 0.7105 - val_accuracy: 0.8582
Epoch 2/100
244/244 [==============================] - 72s 291ms/step - loss: 0.4522 - auc: 0.6314 - accuracy: 0.8443 - val_loss: 0.4166 - val_auc: 0.7102 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 72s 294ms/step - loss: 0.4611 - auc: 0.6235 - accuracy: 0.8498 - val_loss: 0.4189 - val_auc: 0.7293 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 72s 293ms/step - loss: 0.4619 - auc: 0.6327 - accuracy: 0.8523 - val_loss: 0.4205 - val_auc: 0.7247 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 72s 293ms/step - loss: 0.4586 - auc: 0.6389 - accuracy: 0.8497 - val_loss: 0.3869 - val_auc: 0.7240 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 72s 294ms/step - loss: 0.4491 - auc: 0.6486 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 18s 97ms/step - loss: 0.3953 - auc: 0.7478 - accuracy: 0.8592
[0.39531874656677246, 0.747836709022522, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 2

In [ ]:
NAME = "N1024-L2-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 66s 265ms/step - loss: 0.8915 - auc: 0.5562 - accuracy: 0.8077 - val_loss: 0.3647 - val_auc: 0.7616 - val_accuracy: 0.8542
Epoch 2/100
244/244 [==============================] - 63s 255ms/step - loss: 0.3802 - auc: 0.7116 - accuracy: 0.8520 - val_loss: 0.3534 - val_auc: 0.7745 - val_accuracy: 0.8583
Epoch 3/100
244/244 [==============================] - 63s 257ms/step - loss: 0.3640 - auc: 0.7406 - accuracy: 0.8553 - val_loss: 0.3796 - val_auc: 0.7766 - val_accuracy: 0.8572
Epoch 4/100
244/244 [==============================] - 64s 259ms/step - loss: 0.3533 - auc: 0.7600 - accuracy: 0.8580 - val_loss: 0.3529 - val_auc: 0.7893 - val_accuracy: 0.8595
Epoch 5/100
244/244 [==============================] - 64s 261ms/step - loss: 0.3413 - auc: 0.7802 - accuracy: 0.8615 - val_loss: 0.3419 - val_auc: 0.7979 - val_accuracy: 0.8588
Epoch 6/100
244/244 [==============================] - 64s 261ms/step - loss: 0.3368 - auc: 0.7867 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 3

In [ ]:
NAME = "N1024-L2-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 58s 233ms/step - loss: 0.8075 - auc: 0.5481 - accuracy: 0.8177 - val_loss: 0.3998 - val_auc: 0.7314 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 57s 233ms/step - loss: 0.4472 - auc: 0.6494 - accuracy: 0.8372 - val_loss: 0.3795 - val_auc: 0.7374 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 57s 231ms/step - loss: 0.4299 - auc: 0.6671 - accuracy: 0.8477 - val_loss: 0.3963 - val_auc: 0.7585 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 233ms/step - loss: 0.4224 - auc: 0.6888 - accuracy: 0.8511 - val_loss: 0.3523 - val_auc: 0.7690 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 57s 230ms/step - loss: 0.4141 - auc: 0.6929 - accuracy: 0.8530 - val_loss: 0.3665 - val_auc: 0.7758 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3937 - auc: 0.7232 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3428 - auc: 0.7982 - accuracy: 0.8592
[0.34282225370407104, 0.7981623411178589, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 4

In [ ]:
NAME = "N1024-L2-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 57s 225ms/step - loss: 0.9068 - auc: 0.5542 - accuracy: 0.8128 - val_loss: 0.3710 - val_auc: 0.7420 - val_accuracy: 0.8572
Epoch 2/100
244/244 [==============================] - 56s 227ms/step - loss: 0.3786 - auc: 0.7177 - accuracy: 0.8520 - val_loss: 0.3561 - val_auc: 0.7651 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3599 - auc: 0.7485 - accuracy: 0.8576 - val_loss: 0.3457 - val_auc: 0.7782 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 57s 230ms/step - loss: 0.3597 - auc: 0.7501 - accuracy: 0.8570 - val_loss: 0.3479 - val_auc: 0.7775 - val_accuracy: 0.8600
Epoch 5/100
244/244 [==============================] - 56s 229ms/step - loss: 0.3443 - auc: 0.7767 - accuracy: 0.8602 - val_loss: 0.3401 - val_auc: 0.7902 - val_accuracy: 0.8583
Epoch 6/100
244/244 [==============================] - 57s 231ms/step - loss: 0.3348 - auc: 0.7932 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3339 - auc: 0.7976 - accuracy: 0.8690
[0.33392134308815, 0.7975985407829285, 0.8690098524093628]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 5

In [ ]:
NAME = "N1024-L2-D5"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 68s 273ms/step - loss: 0.6799 - auc: 0.5931 - accuracy: 0.8266 - val_loss: 0.6012 - val_auc: 0.7490 - val_accuracy: 0.6481
Epoch 2/100
244/244 [==============================] - 67s 272ms/step - loss: 0.4034 - auc: 0.7053 - accuracy: 0.8411 - val_loss: 0.6863 - val_auc: 0.7307 - val_accuracy: 0.5582
Epoch 3/100
244/244 [==============================] - 67s 273ms/step - loss: 0.3805 - auc: 0.7361 - accuracy: 0.8463 - val_loss: 0.5560 - val_auc: 0.7654 - val_accuracy: 0.6758
Epoch 4/100
244/244 [==============================] - 68s 278ms/step - loss: 0.3656 - auc: 0.7526 - accuracy: 0.8527 - val_loss: 0.9051 - val_auc: 0.7468 - val_accuracy: 0.4317
Epoch 5/100
244/244 [==============================] - 69s 282ms/step - loss: 0.3832 - auc: 0.7335 - accuracy: 0.8457 - val_loss: 0.4066 - val_auc: 0.7654 - val_accuracy: 0.8394
Epoch 6/100
244/244 [==============================] - 68s 276ms/step - loss: 0.3419 - auc: 0.7806 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3558 - auc: 0.7945 - accuracy: 0.8588
[0.35584476590156555, 0.7945382595062256, 0.8588083982467651]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 6

In [ ]:
NAME = "N1024-L2-D6"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 58s 230ms/step - loss: 0.6932 - auc: 0.6147 - accuracy: 0.8302 - val_loss: 0.4832 - val_auc: 0.7743 - val_accuracy: 0.7483
Epoch 2/100
244/244 [==============================] - 57s 232ms/step - loss: 0.3663 - auc: 0.7511 - accuracy: 0.8563 - val_loss: 0.3514 - val_auc: 0.7723 - val_accuracy: 0.8629
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3358 - auc: 0.7918 - accuracy: 0.8649 - val_loss: 0.3679 - val_auc: 0.7896 - val_accuracy: 0.8426
Epoch 4/100
244/244 [==============================] - 57s 233ms/step - loss: 0.3180 - auc: 0.8147 - accuracy: 0.8736 - val_loss: 0.4065 - val_auc: 0.7421 - val_accuracy: 0.8600
Epoch 5/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3057 - auc: 0.8327 - accuracy: 0.8786 - val_loss: 0.3948 - val_auc: 0.7989 - val_accuracy: 0.8194
Epoch 6/100
244/244 [==============================] - 59s 240ms/step - loss: 0.2846 - auc: 0.8545 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3342 - auc: 0.7975 - accuracy: 0.8658
[0.334225594997406, 0.7974523305892944, 0.8658379912376404]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

### L = 3

#### D = 0 [OK]

In [ ]:
NAME = "N512-L3-D0"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 82s 271ms/step - loss: 0.4950 - auc: 0.6255 - accuracy: 0.8388 - val_loss: 0.3647 - val_auc: 0.7815 - val_accuracy: 0.8547
Epoch 2/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3629 - auc: 0.7516 - accuracy: 0.8538 - val_loss: 0.4011 - val_auc: 0.7598 - val_accuracy: 0.8590
Epoch 3/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3329 - auc: 0.7943 - accuracy: 0.8671 - val_loss: 0.3958 - val_auc: 0.8017 - val_accuracy: 0.8101
Epoch 4/100
244/244 [==============================] - 58s 235ms/step - loss: 0.3126 - auc: 0.8211 - accuracy: 0.8768 - val_loss: 0.3360 - val_auc: 0.7933 - val_accuracy: 0.8687
Epoch 5/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3043 - auc: 0.8324 - accuracy: 0.8788 - val_loss: 0.4370 - val_auc: 0.8063 - val_accuracy: 0.7816
Epoch 6/100
244/244 [==============================] - 58s 234ms/step - loss: 0.2926 - auc: 0.8491 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 112ms/step - loss: 0.3268 - auc: 0.8064 - accuracy: 0.8709
[0.32677069306373596, 0.8063855171203613, 0.8708958625793457]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 1 [OK]

In [ ]:
NAME = "N512-L3-D1"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 73s 285ms/step - loss: 1.2899 - auc: 0.5270 - accuracy: 0.7536 - val_loss: 0.4853 - val_auc: 0.6803 - val_accuracy: 0.8572
Epoch 2/100
244/244 [==============================] - 69s 281ms/step - loss: 0.5127 - auc: 0.5970 - accuracy: 0.8058 - val_loss: 0.3840 - val_auc: 0.7371 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 68s 277ms/step - loss: 0.4625 - auc: 0.6441 - accuracy: 0.8381 - val_loss: 0.3976 - val_auc: 0.7437 - val_accuracy: 0.8581
Epoch 4/100
244/244 [==============================] - 68s 276ms/step - loss: 0.4470 - auc: 0.6571 - accuracy: 0.8468 - val_loss: 0.3746 - val_auc: 0.7522 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 68s 276ms/step - loss: 0.4334 - auc: 0.6735 - accuracy: 0.8501 - val_loss: 0.3636 - val_auc: 0.7686 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 68s 277ms/step - loss: 0.4182 - auc: 0.6884 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3557 - auc: 0.7686 - accuracy: 0.8592
[0.35566288232803345, 0.7686297297477722, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 2 [OK]

In [ ]:
NAME = "N512-L3-D2"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 61s 243ms/step - loss: 0.5247 - auc: 0.5750 - accuracy: 0.8338 - val_loss: 0.3768 - val_auc: 0.6819 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 59s 239ms/step - loss: 0.3694 - auc: 0.7259 - accuracy: 0.8568 - val_loss: 0.3423 - val_auc: 0.7890 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3455 - auc: 0.7719 - accuracy: 0.8614 - val_loss: 0.3421 - val_auc: 0.7911 - val_accuracy: 0.8583
Epoch 4/100
244/244 [==============================] - 59s 237ms/step - loss: 0.3344 - auc: 0.7917 - accuracy: 0.8646 - val_loss: 0.3358 - val_auc: 0.8039 - val_accuracy: 0.8669
Epoch 5/100
244/244 [==============================] - 59s 238ms/step - loss: 0.3200 - auc: 0.8150 - accuracy: 0.8685 - val_loss: 0.3267 - val_auc: 0.8113 - val_accuracy: 0.8650
Epoch 6/100
244/244 [==============================] - 59s 239ms/step - loss: 0.3018 - auc: 0.8401 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3521 - auc: 0.8189 - accuracy: 0.8541
[0.35206112265586853, 0.8188831210136414, 0.8540934324264526]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 3 [OK]

In [ ]:
NAME = "N512-L3-D3"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 60s 236ms/step - loss: 0.9955 - auc: 0.5359 - accuracy: 0.8093 - val_loss: 0.4081 - val_auc: 0.6047 - val_accuracy: 0.8581
Epoch 2/100
244/244 [==============================] - 59s 239ms/step - loss: 0.4732 - auc: 0.6126 - accuracy: 0.8352 - val_loss: 0.3691 - val_auc: 0.7494 - val_accuracy: 0.8581
Epoch 3/100
244/244 [==============================] - 58s 236ms/step - loss: 0.4401 - auc: 0.6620 - accuracy: 0.8480 - val_loss: 0.3574 - val_auc: 0.7579 - val_accuracy: 0.8584
Epoch 4/100
244/244 [==============================] - 58s 236ms/step - loss: 0.4225 - auc: 0.6808 - accuracy: 0.8540 - val_loss: 0.3731 - val_auc: 0.7602 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 59s 238ms/step - loss: 0.4244 - auc: 0.6834 - accuracy: 0.8552 - val_loss: 0.3516 - val_auc: 0.7687 - val_accuracy: 0.8581
Epoch 6/100
244/244 [==============================] - 58s 234ms/step - loss: 0.4010 - auc: 0.7096 - accuracy:

In [ ]:
results = model.evaluate(test_data) 
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 98ms/step - loss: 0.3389 - auc: 0.7918 - accuracy: 0.8592
[0.3389175236225128, 0.7917983531951904, 0.859151303768158]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 4 [OK]

In [ ]:
NAME = "N512-L3-D4"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 59s 237ms/step - loss: 0.7742 - auc: 0.5549 - accuracy: 0.8149 - val_loss: 0.3994 - val_auc: 0.6433 - val_accuracy: 0.8578
Epoch 2/100
244/244 [==============================] - 58s 236ms/step - loss: 0.3861 - auc: 0.7002 - accuracy: 0.8547 - val_loss: 0.3549 - val_auc: 0.7683 - val_accuracy: 0.8582
Epoch 3/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3617 - auc: 0.7451 - accuracy: 0.8573 - val_loss: 0.3449 - val_auc: 0.7791 - val_accuracy: 0.8586
Epoch 4/100
244/244 [==============================] - 58s 234ms/step - loss: 0.3490 - auc: 0.7681 - accuracy: 0.8586 - val_loss: 0.3507 - val_auc: 0.7832 - val_accuracy: 0.8581
Epoch 5/100
244/244 [==============================] - 58s 237ms/step - loss: 0.3488 - auc: 0.7708 - accuracy: 0.8582 - val_loss: 0.3400 - val_auc: 0.7883 - val_accuracy: 0.8582
Epoch 6/100
244/244 [==============================] - 58s 236ms/step - loss: 0.3302 - auc: 0.8013 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3540 - auc: 0.7903 - accuracy: 0.8509
[0.3539787530899048, 0.7902542948722839, 0.8509215712547302]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 5 [OK]

In [ ]:
NAME = "N512-L3-D5"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Dropout(0.1))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 dropout (Dropout)           (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 dropout_1 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 71s 283ms/step - loss: 0.5374 - auc: 0.5902 - accuracy: 0.8353 - val_loss: 0.3915 - val_auc: 0.7245 - val_accuracy: 0.8537
Epoch 2/100
244/244 [==============================] - 77s 313ms/step - loss: 0.3775 - auc: 0.7239 - accuracy: 0.8516 - val_loss: 0.4069 - val_auc: 0.7632 - val_accuracy: 0.8467
Epoch 3/100
244/244 [==============================] - 70s 284ms/step - loss: 0.3611 - auc: 0.7490 - accuracy: 0.8559 - val_loss: 0.3527 - val_auc: 0.7763 - val_accuracy: 0.8584
Epoch 4/100
244/244 [==============================] - 69s 279ms/step - loss: 0.3558 - auc: 0.7571 - accuracy: 0.8579 - val_loss: 0.4441 - val_auc: 0.7799 - val_accuracy: 0.8176
Epoch 5/100
244/244 [==============================] - 69s 279ms/step - loss: 0.3463 - auc: 0.7736 - accuracy: 0.8599 - val_loss: 0.5387 - val_auc: 0.7614 - val_accuracy: 0.7070
Epoch 6/100
244/244 [==============================] - 70s 283ms/step - loss: 0.3475 - auc: 0.7721 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 19s 99ms/step - loss: 0.3408 - auc: 0.7942 - accuracy: 0.8668
[0.34078672528266907, 0.7941610217094421, 0.8667809963226318]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

#### D = 6 [OK]

In [ ]:
NAME = "N512-L3-D6"
keras.backend.clear_session()

In [ ]:
ins = image_batch.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=ins))

model.add(Conv2D(32, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Conv2D(64, (3,3), padding="same", strides = (2,2), activation="relu"))

model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(128, activation="relu"))

model.add(Flatten())

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(N, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 32, 32, 128)       8320      
                                                        

In [ ]:
tensorboard = TensorBoard(log_dir="/content/gdrive/MyDrive/Dense Experiments/Logs/{}".format(NAME))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = model.fit(train_data, validation_data=val_data, epochs=100, 
  callbacks=[tensorboard,
              keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/100
244/244 [==============================] - 59s 237ms/step - loss: 0.4872 - auc: 0.6129 - accuracy: 0.8395 - val_loss: 0.4423 - val_auc: 0.7392 - val_accuracy: 0.8179
Epoch 2/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3577 - auc: 0.7537 - accuracy: 0.8600 - val_loss: 0.3466 - val_auc: 0.7936 - val_accuracy: 0.8611
Epoch 3/100
244/244 [==============================] - 59s 240ms/step - loss: 0.3291 - auc: 0.7976 - accuracy: 0.8692 - val_loss: 0.3363 - val_auc: 0.8065 - val_accuracy: 0.8642
Epoch 4/100
244/244 [==============================] - 59s 238ms/step - loss: 0.3078 - auc: 0.8259 - accuracy: 0.8783 - val_loss: 0.3703 - val_auc: 0.7837 - val_accuracy: 0.8622
Epoch 5/100
244/244 [==============================] - 59s 239ms/step - loss: 0.2884 - auc: 0.8509 - accuracy: 0.8878 - val_loss: 0.3901 - val_auc: 0.8096 - val_accuracy: 0.8221
Epoch 6/100
244/244 [==============================] - 59s 240ms/step - loss: 0.2690 - auc: 0.8695 - accuracy:

In [ ]:
results = model.evaluate(test_data)
print(results)

results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)

183/183 [==============================] - 21s 112ms/step - loss: 0.3347 - auc: 0.8046 - accuracy: 0.8633
[0.3347075581550598, 0.8046112656593323, 0.8632661700248718]


In [ ]:
results_pkl = open('/content/gdrive/MyDrive/Dense Experiments/Results/{}.pkl'.format(NAME), 'wb')
pickle.dump(results, results_pkl)